In [22]:
# Imports and variables names
from google.cloud import storage
import pandas as pd
bucket_name = "team4-project"
my_prefix='imgs/euro/'
#Files to be created
full_dataset_name='full_list2.csv' # The full list of images and their catagory
train_dataset_name='train_set2.csv' # 80% to train on
eval_dataset_name='eval_set2.csv' # 10% to eval the training on each epoch
test_dataset_name='test_set2.csv' # 10% to test the model at the end

## Get full List of File Names

In [15]:
storage_client = storage.Client()
bucket = storage_client.get_bucket(bucket_name)
blobs_all = list(bucket.list_blobs(prefix=my_prefix))
gcs_data = []

for x in blobs_all:
    #Select only image files
    if x.name.endswith(('.png','.jpg','.jpeg')):
        gcs_data.append(['gs://'+bucket_name+'/'+x.name,x.name.split('/')[-2]])

## Full Data Out

In [16]:
df = pd.DataFrame(gcs_data)
df2 = df[df[1].isin(['5','10','20','50'])]   
df2.to_csv('text.csv', index = False,header=False)
full_out_path=f'gs://{bucket_name}/{my_prefix}{full_dataset_name}'
!gsutil cp 'text.csv' $full_out_path

Copying file://text.csv [Content-Type=text/csv]...
/ [1 files][ 39.0 KiB/ 39.0 KiB]                                                
Operation completed over 1 objects/39.0 KiB.                                     


## Train Data Out

In [17]:
train_data = df2.sample(frac = 0.8)
train_data.to_csv('train_set.csv', index = False,header=False)
train_out_path = f'gs://{bucket_name}/{my_prefix}{train_dataset_name}.csv'
!gsutil cp 'train_set.csv' $train_out_path



Copying file://train_set.csv [Content-Type=text/csv]...
/ [1 files][ 31.1 KiB/ 31.1 KiB]                                                
Operation completed over 1 objects/31.1 KiB.                                     


## Eval Data Out

In [20]:
# Creating dataframe with rest of the 20% values
non_train_data = df2.drop(train_data.index)
eval_data = non_train_data.sample(frac = 0.5)
eval_data.to_csv('eval_set.csv', index = False,header=False)
eval_out_path=f'gs://{bucket_name}/{my_prefix}{eval_dataset_name}.csv'
!gsutil cp 'eval_set.csv' $eval_out_path

Copying file://eval_set.csv [Content-Type=text/csv]...
/ [1 files][  3.9 KiB/  3.9 KiB]                                                
Operation completed over 1 objects/3.9 KiB.                                      


## Test Data Set Out

In [23]:
# Creating dataframe with rest of the 20% values
test_data = non_train_data.drop(eval_data.index)
test_data.to_csv('test_set.csv', index = False,header=False)
test_out_path=f'gs://{bucket_name}/{my_prefix}{test_dataset_name}.csv'
!gsutil cp 'test_set.csv' $test_out_path

Copying file://test_set.csv [Content-Type=text/csv]...
/ [1 files][  4.0 KiB/  4.0 KiB]                                                
Operation completed over 1 objects/4.0 KiB.                                      
